In [ ]:
import calendar
import re
import time

import pandas as pd

In [ ]:
data = pd.read_csv("combined.csv")
data["Descriptipn"] = data["Descriptipn"].values.astype('str')

data.head()

In [ ]:
data["Emails"] = ""

for i, j in data.iterrows():
    emails = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", j.Descriptipn)
    data.at[i, "Emails"] = emails

In [ ]:
data["Numbers"] = ""
whitelist = set("1234567890 +,-()|")

for i, j in data.iterrows():
    data.at[i, "Numbers"] = "".join(filter(whitelist.__contains__, j.Descriptipn))

In [ ]:
for i, j in data.iterrows():
    if not data.at[i, "Emails"] and sum(1 for char in data.at[i, "Numbers"] if char.isdigit()) < 10:
        data.drop(i, inplace=True)

In [ ]:
data["Niche"] = ""

for i, j in data.iterrows():
    if pd.isna(j["Keyword"]):
        data.at[i, "Niche"] = "Null"
    else:
        niche = j["Keyword"].split("+ ")[1]
        data.at[i, "Niche"] = niche

In [ ]:
data["Type"] = ""

for i, j in data.iterrows():
    if "linkedin" in j["Detail_URL"]:
        link_type = j["Detail_URL"].split("linkedin")[1].split("/")[1]
        match link_type:
            case "in":
                data.at[i, "Type"] = "Candidate/Person"
            case "posts":
                data.at[i, "Type"] = "Post"
            case "company":
                data.at[i, "Type"] = "Company/Agency"
            case "jobs":
                data.at[i, "Type"] = "Job Post"
            case _:
                data.at[i, "Type"] = "Other"
    elif "indeed" in j["Detail_URL"]:
        data.at[i, "Type"] = "Indeed"
    else:
        data.at[i, "Type"] = "Other"

In [ ]:
data.drop(columns=["Language", "Source", "ErrorMessage", "Keyword"], inplace=True)
data.head(n=100)

In [ ]:
data.to_csv("CleanedData " + str(calendar.timegm(time.gmtime())) + ".csv", index=False)